In [6]:
from constants import *
import glob
import pandas as pd

In [7]:
filenames = glob.glob("./Pass1_outputs/*")

In [8]:
def df_maker(filepath : str):
    with open(filepath, 'r') as fp:
        text = fp.read()

    text = text.split("\n")
    if "effectiveness" in filepath:
        text = [t.split(": ") for t in text]
    else:
        text = [t.split(" : ") for t in text]

    df = {
        'Use Case ID' : [],
        'Ranks' : []
    }

    for t in text:
        df['Use Case ID'].append(t[0].strip())
        df['Ranks'].append(t[1].strip())


    df = pd.DataFrame.from_dict(df, orient = 'index').transpose()
    
    return df

In [9]:
dfs = []

for fp in filenames:
    print(fp)
    rank_class = fp.split("/")[2].split("_")[0].upper()
    rank_class = rank_class + "_Ranking"
    
    df = df_maker(fp)
    df.rename(columns= {"Ranks" : rank_class}, inplace=True)

    dfs.append(df)

./Pass1_outputs/bias_pass1.txt
./Pass1_outputs/complexity_pass1.txt
./Pass1_outputs/expensiveness_pass1.txt
./Pass1_outputs/explainable_pass1.txt
./Pass1_outputs/safe_pass1.txt
./Pass1_outputs/trustworthy_pass1.txt
./Pass1_outputs/effectiveness_pass1.txt


In [10]:
def join_all(dfs):
    start_df = dfs[0]

    for i in range(1, len(dfs)):
        start_df = start_df.merge(dfs[i], on="Use Case ID")

    return start_df

In [11]:
final_df = join_all(dfs)

In [12]:
main_df = pd.read_excel("./Data/output.xlsx", engine='openpyxl')

In [13]:
final_df_cols = list(final_df.columns)

In [14]:
final_df_cols

['Use Case ID',
 'BIAS_Ranking',
 'COMPLEXITY_Ranking',
 'EXPENSIVENESS_Ranking',
 'EXPLAINABLE_Ranking',
 'SAFE_Ranking',
 'TRUSTWORTHY_Ranking',
 'EFFECTIVENESS_Ranking']

In [15]:
final_df = final_df.merge(main_df, on = "Use Case ID")

In [16]:
final_df.shape

(58, 29)

In [17]:
for i in range(final_df.shape[0]):
    final_df['SAFE_Ranking'][i] = str(int(final_df['SAFE_Ranking'][i]) - 1)


/tmp/ipykernel_88554/2569430546.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  final_df['SAFE_Ranking'][i] = str(int(final_df['SAFE_Ranking'][i]) - 1)
/tmp/ipykernel_88554/2569430546.py:2: SettingWithCopyWarning: 
A value is trying to b

In [18]:
final_df = final_df[final_df_cols + ["Use Case Name", "gpt-4-turbo-preview_summaries"]]

In [19]:
final_df.to_csv("./Data/Streamlit_pass1.csv", index = False)

In [20]:
final_df.columns

Index(['Use Case ID', 'BIAS_Ranking', 'COMPLEXITY_Ranking',
       'EXPENSIVENESS_Ranking', 'EXPLAINABLE_Ranking', 'SAFE_Ranking',
       'TRUSTWORTHY_Ranking', 'EFFECTIVENESS_Ranking', 'Use Case Name',
       'gpt-4-turbo-preview_summaries'],
      dtype='object')

In [23]:
final_df["EFFECTIVENESS_Ranking"].unique()

array(['58', '56', '54', '52', '50', '48', '46', '44', '42', '40', '38',
       '36', '34', '32', '30', '28', '26', '24', '22', '20', '18', '16',
       '14', '12', '10', '8', '6', '4', '2', '1', '3', '5', '7', '9',
       '11', '13', '15', '17', '19', '21', '23', '25', '27', '29', '31',
       '33', '35', '37', '39', '41', '43', '45', '47', '49', '51', '53',
       '55', '57'], dtype=object)